In [ ]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-78cbg7yz
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-78cbg7yz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 32.5 MB/s 
     |████████████████████████████████| 531 kB 42.7 MB/s 
     |████████████████████████████████| 269 kB 50.3 MB/s 
     |████████████████████████████████| 180 kB 41.8 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 133 kB 53.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=410388 sha256=8767dc1f9ef1adb58f32433cf7fdc3de98b0c97270c75ff54be8249396c72c15
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dgenwt3/wheels/60/fb/ef/c106a98bc821c758882e32e24bb6b3ed11b6917155

In [ ]:
from avalanche.benchmarks.classic.ccifar10 import SplitCIFAR10
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
from avalanche.models import SimpleMLP,IcarlNet,IncrementalClassifier,MTSimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive,LwF,EWC

from torch.optim import SGD
from torch.nn import CrossEntropyLoss,NLLLoss

In [ ]:
from avalanche.benchmarks.datasets import CIFAR10

In [ ]:
from avalanche.models import IcarlNet

In [ ]:
from avalanche.models import SimpleMLP

In [ ]:
import torch

In [ ]:

device = 'cpu' if torch.cuda.is_available() else 'cpu'
scenario = SplitCIFAR10(n_experiences=5,return_task_id=True)
scenario2 = SplitCIFAR10(n_experiences=5,return_task_id=True)
from avalanche.models import SimpleCNN
from avalanche.models import SimpleMLP_TinyImageNet


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/cifar10/cifar-10-python.tar.gz to /root/.avalanche/data/cifar10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = IcarlNet(num_classes=2, n=5, c=3).to(device)
model2 = IcarlNet(num_classes=2, n=5, c=3).to(device)


In [ ]:
from torchsummary import summary

In [ ]:
summary(model,(3,32,32))
summary(model2,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
          Identity-9           [-1, 16, 32, 32]               0
    ResidualBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,

In [ ]:
tb_logger = TensorboardLogger()
tb_logger2 = TensorboardLogger(tb_log_dir="./tb_data2")
int_logger = InteractiveLogger()
int_logger2 = InteractiveLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=False, experience=True),
    loss_metrics(minibatch=False, stream=False),
    forgetting_metrics(experience=True, stream=True), 
    loggers=[int_logger, tb_logger])
eval_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=False, experience=True),
    loss_metrics(minibatch=False, stream=False),
    forgetting_metrics(experience=True, stream=True), 
    loggers=[int_logger2, tb_logger2])

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


In [ ]:
for i in range(1,2):
  print(f'{10**-i} Learning Rate')
  cl_strategy = EWC(
      model, SGD(model.parameters(), lr=10**(-i), momentum=0.9),
      CrossEntropyLoss(), ewc_lambda=.4, train_mb_size=500, train_epochs=5, eval_mb_size=100,
      evaluator=eval_plugin)
  cl_strategy2 = EWC(
      model2, SGD(model2.parameters(), lr=10**(-i), momentum=0.9),
      CrossEntropyLoss(), ewc_lambda=.4, train_mb_size=500, train_epochs=5, eval_mb_size=100,
      evaluator=eval_plugin2)
  # cl_strategy = EWC(model, SGD(model.parameters,lr=10**(-i), momentum=.9), CrossEntropyLoss(), ewc_lambda=.4,
  #                   train_mb_size=500, train_epochs=5, eval_mb_size=100,
  #                    evaluator=eval_plugin)

  # TRAINING LOOP
print('Starting experiment...')
results = []
for train_batch_info in scenario.train_stream:
  train_batch_info
  print("Start training on experience ", train_batch_info.current_experience)
  cl_strategy.train(train_batch_info, num_workers=4)
  print("End training on experience ", train_batch_info.current_experience)
  print('Computing accuracy on the test set')
  results.append(cl_strategy.eval(scenario.test_stream[:]))

print('Starting experiment...')
results2 = []
for train_batch_info in scenario2.train_stream:
  train_batch_info
  print("Start training on experience ", train_batch_info.current_experience)
  cl_strategy2.train(train_batch_info, num_workers=4)
  print("End training on experience ", train_batch_info.current_experience)
  print('Computing accuracy on the test set')
  results.append(cl_strategy2.eval(scenario2.test_stream[:]))

0.1 Learning Rate
Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [02:28<00:00,  7.43s/it]
Epoch 0 ended.
100%|██████████| 20/20 [02:28<00:00,  7.43s/it]
Epoch 1 ended.
100%|██████████| 20/20 [02:29<00:00,  7.47s/it]
Epoch 2 ended.
100%|██████████| 20/20 [02:32<00:00,  7.62s/it]
Epoch 3 ended.
100%|██████████| 20/20 [02:27<00:00,  7.38s/it]
Epoch 4 ended.
-- >> End of training phase << --
End training on experience  0
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 20/20 [00:08<00:00,  2.32it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8815
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 20/20 [00:08<00:00,  2.24it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.2930
-- Starting eval on experience 2 (Task 2) from test stream --
100%|██████████| 20/20 [00:08<00:00,  